In [1]:
import requests
from bs4 import BeautifulSoup

import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta
from tqdm import tqdm
import time
import re

In [2]:
#DataFrame에 들어갈 list 생성
dates = []
category = []
press = []
titles = []
documents = []
link = []

In [3]:
# 시작, 종료 날짜 현재 기준으로 설정, 페이지값 초기화(이건 나중에 옮기기)
curr_date = datetime.now().strftime("%Y%m%d")
end_date = (datetime.now() - relativedelta(days=3)).strftime("%Y%m%d")
page = 1
count = 1

In [4]:
def call_source(search_date, search_page):
    url = "https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid2=230&sid1=105&date=" + search_date + "&page=" + str(search_page)
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
    web = requests.get(url, headers=headers).content
    source = BeautifulSoup(web, 'html.parser')
    
    return source

In [5]:
def scrap_news(url, dates, category, press, titles, documents, link):
    try:
        content = requests.get(url).content
        source = BeautifulSoup(content, 'html.parser')

        # 기사날짜
        date = source.find('span', {'class': 'media_end_head_info_datestamp_time _ARTICLE_DATE_TIME'}).get_text()

        # 언론사
        company = source.find('em', {'class' : 'media_end_linked_more_point'}).get_text()
        
        # 기사제목
        title = source.find('h2').get_text()

        # 기사본문
        article = source.find('article', {'id': 'dic_area'}).get_text()
        news_content = article.replace("\n", "")
        news_content = news_content.replace("// flash 오류를 우회하기 위한 함수 추가function _flash_removeCallback() {}", "")
        news_content = news_content.replace("동영상 뉴스       ", "")
        news_content = news_content.replace("동영상 뉴스", "")
        news_content = news_content.strip()

        dates.append(date)
        category.append("IT 일반")
        press.append(company)
        titles.append(title)
        documents.append(news_content)
        link.append(url)
        
    except:
        pass
    

In [6]:
# Main Code
while curr_date != end_date:
    # page 값 초기화
    page = 1
    
    # max page 구하기
    source = call_source(curr_date, 100)
    max_page = int(source.find_all('strong')[2].get_text())
    max_page += 1 # range 함수 고려
    
    for pg in tqdm(range(1, max_page)):
        # 현재 페이지 정보 불러오기
        source = call_source(curr_date, page)
        page_list = source.find_all('a', {'class' : 'nclicks(fls.page)'})

        # 각 기사별 링크 추출
        article_list = []
        article_set = source.find_all('dt')

        for info in article_set:
            url = info.find('a')['href']
            article_list.append(url)

        article_list = list(dict.fromkeys(article_list))

        # 각 기사별 스크랩 진행
        for url in article_list:
            scrap_news(url, dates, category, press, titles, documents, link)

        # 다음 페이지
        page += 1
        
    # 날짜, page 초기화
    curr_date = (datetime.now() - relativedelta(days=count)).strftime("%Y%m%d")
    count += 1

100%|██████████████████████████████████████████████████████████████████████████████████| 26/26 [09:37<00:00, 22.20s/it]


In [7]:
article_df = pd.DataFrame({'Date':dates,
                          'Category':category,
                          'Press':press,
                          'Title':titles,
                          'Document':documents,
                          'Link':link})

article_df.to_excel('result_{}.xlsx'.format(datetime.now().strftime('%y%m%d_%H%M')), index=False)
article_df.head()

,Date,Category,Press,Title,Document,Link
0,2023.10.06. 오전 11:23,IT 일반,연합뉴스,"[게시판] 코스콤, 롯데카드 클라우드 컨택센터 구축 완료",[코스콤 제공. 재판매 및 DB 금지] ▲ 코스콤은 네이버클라우드와 함께 최근...,https://n.news.naver.com/mnews/article/001/001...
1,2023.10.06. 오전 11:23,IT 일반,이데일리,"동아ST-GC녹십자, 면역질환 신약개발에 머리 맞댄다",[이데일리 나은경 기자] 동아에스티(170900)는 GC녹십자(006280)와 면역...,https://n.news.naver.com/mnews/article/018/000...
2,2023.10.06. 오전 11:23,IT 일반,지디넷코리아,"""4년간 연구했다""…하림 '더미식 육즙만두' 맛보니","""얇고 쫄깃한 만두피·촉촉한 육즙 합격""...냉동만두 시장 10% 점유 목표""4년간...",https://n.news.naver.com/mnews/article/092/000...
3,2023.10.06. 오전 11:22,IT 일반,헤럴드경제,"DGIST 학생 창업기업, 창업아이디어 경진대회 석권",이정우(오른쪽) 퀘스터 대표와 김규민 Friending 대표.[DGIST 제공][헤...,https://n.news.naver.com/mnews/article/016/000...
4,2023.10.06. 오전 11:21,IT 일반,헤럴드경제,"사우디 통신정보기술부 장관, 네이버 1784 방문...미래기술 논의",AI·디지털트윈 기술협력 가능성5일 네이버 1784에 방문한 사우디아라비아 통신정보...,https://n.news.naver.com/mnews/article/016/000...
